In [3]:
import json

def calculate_token_cost(token_usage_list, input_rate=2.5, output_rate=10, num_q=500):
    """
    計算總成本（以新台幣計算），根據 token 使用量。

    Args:
        token_usage_list (list of dict): 每個 dict 包含 prompt_tokens, completion_tokens
        input_rate (float): 輸入成本，單位為每 1M token 的新台幣（預設 2.5）
        output_rate (float): 輸出成本，單位為每 1M token 的新台幣（預設 10）
        num_q (int): 這個參數在此聚合用法中可能需要調整，見下方說明。

    Returns:
        dict: 包含輸入、輸出、總成本與總 token
    """

    # 使用 .get() 避免缺少鍵值的錯誤，並提供預設值 0
    total_prompt = sum(x.get("prompt_tokens", 0) for x in token_usage_list)
    total_completion = sum(x.get("completion_tokens", 0) for x in token_usage_list)

    # --- 重要說明關於 num_q ---
    # 這個函數的原始設計似乎是針對 *單個* 查詢（可能包含多個步驟/API調用）的 token 列表，
    # 然後乘以 num_q (查詢數量) 來得到總成本。
    # 但在這裡，我們是先從多個 (約500個) 查詢聚合了 *所有* 的 token_usage。
    # 因此，total_prompt 和 total_completion 已經是 *總和*。
    # 如果我們再乘以 num_q (例如 500)，成本會被錯誤地放大 500 倍。
    # 所以，當使用已聚合的列表調用此函數時，應將 num_q 設為 1。
    # 如果您確定需要將總 token 數再乘以文件中的項目數，請取消下面一行的註解。
    # input_cost = total_prompt * num_q * input_rate / 1000000
    # output_cost = total_completion * num_q * output_rate / 1000000

    # 使用 num_q = 1 進行計算，因為 token_usage_list 已包含所有數據
    input_cost = total_prompt * 1 * input_rate / 1000000
    output_cost = total_completion * 1 * output_rate / 1000000
    total_cost = input_cost + output_cost

    return {
        "total_prompt_tokens": total_prompt,
        "total_completion_tokens": total_completion,
        "input_cost_ntd": round(input_cost, 2),
        "output_cost_ntd": round(output_cost, 2),
        "total_cost_ntd": round(total_cost, 2)
    }

# --- 主程式邏輯 ---
file_path = 'CQ_Solver_summary.json'
all_token_usage_entries = []

try:
    # 1. 讀取 JSON 文件
    with open(file_path, 'r', encoding='utf-8') as f:
        # 假設 react_summary.json 包含一個 JSON 對象列表
        data = json.load(f)

    # 檢查加載的數據是否為列表
    if isinstance(data, list):
        # 2. 提取所有 token_usage 字典並合併到一個列表中
        for item in data:
            # 檢查 item 是否為字典且包含 'token_usage' 鍵，其值為列表
            if isinstance(item, dict) and "token_usage" in item and isinstance(item["token_usage"], list):
                all_token_usage_entries.extend(item["token_usage"])
            else:
                # 可以選擇性地打印警告信息
                # print(f"警告：跳過格式不符的項目：{item}")
                pass
        print(f"從 {len(data)} 個項目中成功提取並合併了 {len(all_token_usage_entries)} 個 token_usage 記錄。")

    else:
        print(f"錯誤：JSON 文件 '{file_path}' 的頂層結構不是預期的列表，而是 {type(data)}。")
        # 根據需要決定是否退出或進行其他處理
        exit() # 或者 raise TypeError("JSON 頂層結構錯誤")

    # 3. 使用聚合後的列表計算成本
    #    注意：我們在這裡傳入 num_q=1，因為 all_token_usage_entries 已包含所有數據
    #    使用您範例中提供的費率 (input_rate=0.59, output_rate=0.79)
    costs = calculate_token_cost(
        token_usage_list=all_token_usage_entries,
        input_rate=0.59,
        output_rate=0.79,
        num_q=1 # 關鍵：因為列表已聚合，所以 num_q 設為 1
    )

    # 4. 打印結果
    print("\n--- 計算結果 ---")
    print(f"總 Prompt Tokens: {costs['total_prompt_tokens']}")
    print(f"總 Completion Tokens: {costs['total_completion_tokens']}")
    print(f"輸入成本 (NTD): {costs['input_cost_ntd']}")
    print(f"輸出成本 (NTD): {costs['output_cost_ntd']}")
    print(f"總成本 (NTD): {costs['total_cost_ntd']}")

except FileNotFoundError:
    print(f"錯誤：找不到文件 '{file_path}'。請確認文件路徑是否正確。")
except json.JSONDecodeError:
    print(f"錯誤：無法解析 '{file_path}' 的 JSON 內容。請檢查文件格式是否正確。")
except Exception as e:
    print(f"發生未預期的錯誤：{e}")

從 10 個項目中成功提取並合併了 162 個 token_usage 記錄。

--- 計算結果 ---
總 Prompt Tokens: 605578
總 Completion Tokens: 30080
輸入成本 (NTD): 0.36
輸出成本 (NTD): 0.02
總成本 (NTD): 0.38
